In [4]:
# import required packages
import pandas as pd
import requests
import hashlib
import json
import string

In [5]:
# function to create characters dataframe by making API call
def create_character_df (input_apikey, input_hash, nameStartWith):
    ts = 1
    address = 'http://gateway.marvel.com/v1/public/characters'
    df = pd.DataFrame()

    #looping to get all characters starting with certain character, when there are more than 100 suitable characters - using offset
    for i in range(0,3):
        parameters = {
            'apikey':input_apikey,
            'ts':ts,
            'hash':input_hash,
            'limit':100,
            'nameStartsWith':nameStartWith,
            'offset':100*i
        }
        response = requests.get(address, params=parameters)
        results = response.json()                             
        response.raise_for_status() # raise exception if http error
        df_new = pd.json_normalize(results['data']['results'])
        df = pd.concat([df, df_new], ignore_index=True)
    character_df = df[['id','name','events.available','series.available','stories.available','comics.available']]
    return(character_df)


In [6]:
public_key = '486e33cb6586570842beb1fdc21db6c2'
private_key = '4893102e65894f0350a76693dff1b2bc9418a52a'
ts = 1
hash = hashlib.md5((str(ts)+private_key+public_key).encode()).hexdigest()

In [7]:
char_df = create_character_df(input_apikey=public_key, input_hash=hash, nameStartWith='S')
char_df

,id,name,events.available,series.available,stories.available,comics.available
0,1009552,S.H.I.E.L.D.,1,72,196,165
1,1009553,Sabra,0,13,21,21
2,1009554,Sabretooth,8,113,326,309
3,1010969,Sabretooth (Age of Apocalypse),1,7,21,21
4,1010970,Sabretooth (House of M),1,1,1,1
...,...,...,...,...,...,...
197,1017852,Sword Master (Sword Master),0,7,25,25
198,1009641,Swordsman,4,25,56,60
199,1010982,Swordsman (Jacques Duquesne),1,2,4,4
200,1009642,Sym,0,1,3,2


In [27]:
# function to filter the characters dataframe
def filter_characters(df, column_name, filter_condition, filter_value):
    if column_name == 'name':
        filter_condition = 'starts_with'
        length =len(filter_value)
        return(df[df.name.str[:length] == filter_value])
    else:
        if filter_condition == 'equal_to':
            return(df[df[column_name] == int(filter_value)])
        elif filter_condition == 'less_than':
            return(df[df[column_name] < int(filter_value)])
        elif filter_condition == 'greater_than':
            return(df[df[column_name] > int(filter_value)])
        else:
            return('ERROR - Invalid filter condition')

In [30]:
# trying out different filters to test the function

result = filter_characters(char_df, 'events.available', 'greater_than', 1)
#result = filter_characters(char_df, 'series.available', 'equal_to', 22)
#result = filter_characters(char_df, 'name', 'starts_with', 'Sa')
#result = filter_characters(char_df, 'events.available', 'more_than', 1)
result

,id,name,events.available,series.available,stories.available,comics.available
31,1009581,Sebastian Shaw,5,22,42,41
82,1009591,Silver Samurai,0,22,32,39
139,1009614,Spiral (Rita Wayword),4,22,52,50
188,1009639,Super-Skrull,3,22,35,31
